In [5]:
import numpy as np
from PIL import Image
import torch
from torch.utils.data import DataLoader
from PytorchWildlife.models import detection as pw_detection
from PytorchWildlife.data import transforms as pw_trans
from PytorchWildlife.data import datasets as pw_data 
from PytorchWildlife import utils as pw_utils
import os

In [2]:
torch.cuda.is_available()

True

In [6]:
# Setting the device to use for computations ('cuda' indicates GPU)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
detection_model = pw_detection.MegaDetectorV5(device=DEVICE, pretrained=True)

Fusing layers... 
Fusing layers... 
Model summary: 733 layers, 140054656 parameters, 0 gradients, 208.8 GFLOPs
Model summary: 733 layers, 140054656 parameters, 0 gradients, 208.8 GFLOPs


In [22]:
tgt_folder_path = os.path.join(".","D:\C_Drive\Pictures\IIITD Dataset","gaze_benn-Gazella bennettii")
dataset = pw_data.DetectionImageFolder(
    tgt_folder_path,
    transform=pw_trans.MegaDetector_v5_Transform(target_size=detection_model.IMAGE_SIZE, stride=detection_model.STRIDE),
    extension='jpg' # Can be other extensions based on your own data, such as png. And this argument is case sensitive.
)
loader = DataLoader(dataset, batch_size=32, shuffle=False, 
                    pin_memory=True, num_workers=0, drop_last=False)
results = detection_model.batch_image_detection(loader)

100%|██████████| 16/16 [12:06<00:00, 45.44s/it]


In [23]:
pw_utils.save_detection_json(results, os.path.join(".","annotations/batch_output_6.json"),
                            categories=detection_model.CLASS_NAMES,
                            exclude_category_ids=[], # Category IDs can be found in the definition of each model.
                            exclude_file_path=None)

In [24]:
pw_utils.save_detection_images(results, "batch_output6", overwrite=False)

In [25]:
import json
import os
from PIL import Image  # Use PIL library to get image dimensions

def process_json_and_save(json_file, output_folder):
    with open(json_file, 'r') as f:
        data = json.load(f)
        
    annotations_per_image = {}
    
    for annotation in data['annotations']:
        img_id = annotation['img_id']
        categories = annotation['category']
        confidences = annotation['confidence']
        
        img_filename = os.path.splitext(os.path.basename(img_id))[0]
        
        if img_filename not in annotations_per_image:
            annotations_per_image[img_filename] = []
        
        try:
            img = Image.open(img_id)
            img_width, img_height = img.size
        except Exception as e:
            print(f"Error processing image {img_id}: {e}")
            continue
        
        for i, bbox in enumerate(annotation['bbox']):
            category = categories[i]
            confidence = confidences[i]
            
            left, top, width, height = bbox
            relative_left = left / img_width
            relative_top = top / img_height
            relative_width = width / img_width
            relative_height = height / img_height
            
            annotation_str = f"{5} {relative_left:.6f},{relative_top:.6f},{relative_width:.6f},{relative_height:.6f} {confidence:.6f}"
            annotations_per_image[img_filename].append(annotation_str)
    
    for img_filename, annotations_list in annotations_per_image.items():
        os.makedirs(output_folder, exist_ok=True)
        
        output_filename = os.path.join(output_folder, f"{img_filename}.txt")
        with open(output_filename, 'w') as outfile:
            for annotation_str in annotations_list:
                outfile.write(annotation_str + '\n')
        
        print(f"Saved annotations for {img_filename}.txt")

json_file_path = 'batch_output_6.json'
output_folder_path = 'D:\C_Drive\Pictures\IIITD Dataset/gaze_benn-Gazella bennettii_labels'

process_json_and_save(json_file_path, output_folder_path)

Saved annotations for ind_rant_04_2014_001_P035_76509917_26102833_c012b_02_06_2014_11_07_34_P_1121.txt
Saved annotations for ind_rant_04_2014_001_P035_76509917_26102833_c012b_03_06_2014_12_42_06_P_1141.txt
Saved annotations for ind_rant_04_2014_001_P036_76513222_26097806_c012c_06_06_2014_12_57_14_P_10.txt
Saved annotations for ind_rant_04_2014_001_P036_76513222_26097806_c012c_06_06_2014_12_57_55_P_11.txt
Saved annotations for ind_rant_04_2014_001_P036_76513222_26097806_c012c_07_06_2014_06_51_23_P_20.txt
Saved annotations for ind_rant_04_2014_001_P036_76513222_26097806_c012c_10_06_2014_13_02_39_P_122.txt
Saved annotations for ind_rant_04_2014_001_P052_76599361_26103889_c018a_06_06_2014_14_24_30_P_48.txt
Saved annotations for ind_rant_04_2014_001_P052_76599361_26103889_c018a_06_06_2014_14_24_52_P_52.txt
Saved annotations for ind_rant_04_2014_001_P052_76599361_26103889_c018a_09_06_2014_09_36_52_P_69.txt
Saved annotations for ind_rant_04_2014_001_P052_76599361_26103889_c018a_09_06_2014_09_